### Stock Analysis

In this part we will make analysis using RSI, volatility and other basic financial tools.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Creating a function to retrieve financial data from yfinance.

In [2]:
def get_stock_data(ticker, period='1y'):
    df = yf.download(ticker, period=period, auto_adjust=True)
    
    # Sütun isimlerindeki karmaşıklığı (MultiIndex) temizle
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    
    # Getirileri Hesapla
    # Log Getiri (Analiz ve Volatilite için daha doğru)
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    return df

Creating a function that generates technical indicators.

In [3]:
def add_indicators(df):
    # Simple Moving Average (SMA 20 and 50)
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # RSI (Relative Strength Index)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    return df

Creating a function to generate plot-candlestick

In [4]:
def plot_dashboard(df, ticker):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=(f'{ticker} Price Analysis', 'RSI Momentum'),
                        row_width=[0.2, 0.7])

    # Candlestick Graph
    fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'], name='Fiyat'), row=1, col=1)

    # SMA Lines
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], line=dict(color='orange', width=1), name='SMA 20'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], line=dict(color='blue', width=1), name='SMA 50'), row=1, col=1)

    # 2nd Row: RSI
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], line=dict(color='purple', width=2), name='RSI'), row=2, col=1)
    
    # Reference lines for RSI at 70 and 30
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    # Layout adjustments
    fig.update_layout(title=f'{ticker} Technical Analysis Dashboard', yaxis_title='Price', xaxis_rangeslider_visible=False, height=800)
    fig.show()

## Using the functions:

In [5]:
symbol = "GARAN.IS" # Or "AAPL", "NVDA"

# 1. Get Data
data = get_stock_data(symbol)
print(data.head())

# 2. Add Indicators
data = add_indicators(data)

# 3. Plot Dashboard
plot_dashboard(data, symbol)

# 4. Show Last 5 Days
print(data[['Close', 'Log_Return', 'RSI']].tail())

[*********************100%***********************]  1 of 1 completed


Price            Close        High         Low        Open    Volume  \
Date                                                                   
2024-12-12  121.314445  123.243133  120.350102  122.375222  18260905   
2024-12-13  123.243134  123.243134  119.192885  120.060796  17103441   
2024-12-16  123.243134  125.075381  122.085917  123.243134  21133440   
2024-12-17  125.846855  126.811199  122.278786  123.532436  22313133   
2024-12-18  123.821732  126.811203  123.821732  125.268247  19049800   

Price       Log_Return  
Date                    
2024-12-12         NaN  
2024-12-13    0.015773  
2024-12-16    0.000000  
2024-12-17    0.020907  
2024-12-18   -0.016223  


Price            Close  Log_Return        RSI
Date                                         
2025-12-08  141.199997   -0.002122  71.428548
2025-12-09  140.600006   -0.004258  65.873008
2025-12-10  139.699997   -0.006422  59.051707
2025-12-11  139.199997   -0.003586  64.018676
2025-12-12  140.399994    0.008584  62.801895


## Volatility


In [6]:
def volatility_analysis(df):
    volatility = df['Log_Return'].rolling(window=21).std() * np.sqrt(252)  # Annualized Volatility
    df['Volatility'] = volatility
    return df

def year_volality_analysis(df):
    yearly_volatility = df['Log_Return'].resample('Y').std() * np.sqrt(252)
    df['Yearly_Volatility'] = yearly_volatility
    return df

# Volatility Analysis for multiple symbols
symbol1 = "NVDA"
data1 = get_stock_data(symbol1)
data1 = add_indicators(data1)

# Volatility Analysis for symbol1
data1 = volatility_analysis(data1)
print(f"Volatility Analysis for {symbol1}:")
print(data1[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())

symbol2 = "AAPL"
data2 = get_stock_data(symbol2)
data2 = add_indicators(data2)

# Volatility Analysis for symbol2
data2 = volatility_analysis(data2)
print(f"Volatility Analysis for {symbol2}:")
print(data2[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())

symbol3 = "GARAN.IS"
data3 = get_stock_data(symbol3)
data3 = add_indicators(data3)
# Volatility Analysis for symbol3
data3 = volatility_analysis(data3)
print(f"Volatility Analysis for {symbol3}:")
print(data3[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Volatility Analysis for NVDA:
Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  185.550003    0.017067    0.382635  48.879194
2025-12-09  184.970001   -0.003131    0.382711  54.338761
2025-12-10  183.779999   -0.006454    0.321538  46.384468
2025-12-11  180.929993   -0.015629    0.310405  50.432113
2025-12-12  175.020004   -0.033210    0.326260  45.047403



[*********************100%***********************]  1 of 1 completed

Volatility Analysis for AAPL:
Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  277.890015   -0.003198    0.167281  66.456347
2025-12-09  277.179993   -0.002558    0.166387  65.040131
2025-12-10  278.779999    0.005756    0.166714  65.550816
2025-12-11  278.029999   -0.002694    0.150891  68.817878
2025-12-12  278.279999    0.000899    0.148748  62.903841
Volatility Analysis for GARAN.IS:
Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  141.199997   -0.002122    0.268843  71.428548
2025-12-09  140.600006   -0.004258    0.249783  65.873008
2025-12-10  139.699997   -0.006422    0.239410  59.051707
2025-12-11  139.199997   -0.003586    0.239618  64.018676
2025-12-12  140.399994    0.008584    0.240230  62.801895


## Package

In [ ]:
# Creating packaged analysis function
def analysis(symbol, period='1y'):
    data = get_stock_data(symbol, period=period)
    data = add_indicators(data)
    data = volatility_analysis(data)
    return data

AAPL_data = analysis("AAPL", "10y")
print(AAPL_data[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())
# AAPL_data.to_csv("AAPL_analysis.csv")

plot_dashboard(AAPL_data, "AAPL")

[*********************100%***********************]  1 of 1 completed

Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  277.890015   -0.003198    0.167281  66.456347
2025-12-09  277.179993   -0.002558    0.166387  65.040131
2025-12-10  278.779999    0.005756    0.166714  65.550816
2025-12-11  278.029999   -0.002694    0.150891  68.817878
2025-12-12  278.279999    0.000899    0.148748  62.903841


## Expected Returns and Covariance Matrix

In [8]:
def get_portfolio_history(symbols, period='1y'):
    portfolio_df = pd.DataFrame()
    
    for symbol in symbols:
        data = get_stock_data(symbol, period=period) 
        portfolio_df[symbol] = data['Log_Return']
        
    return portfolio_df


symbols = ["AAPL", "NVDA", "GARAN.IS"]


df_portfolio = get_portfolio_history(symbols)

expected_returns = df_portfolio.mean() * 252

covariance_matrix = df_portfolio.cov() * 252

print("--- Beklenen Yıllık Getiriler ---")
print(expected_returns)
print("\n--- Kovaryans Matrisi (Risk İlişkisi) ---")
print(covariance_matrix)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

--- Beklenen Yıllık Getiriler ---
AAPL        0.120624
NVDA        0.268668
GARAN.IS    0.064886
dtype: float64

--- Kovaryans Matrisi (Risk İlişkisi) ---
              AAPL      NVDA  GARAN.IS
AAPL      0.105281  0.067297  0.010511
NVDA      0.067297  0.248530  0.017397
GARAN.IS  0.010511  0.017397  0.149207


## Portfolio Volality & Return

In [9]:
symbols = ["AAPL", "NVDA", "INTC"]
weights = [0.4, 0.4, 0.2] # Example Weights

def portfolio_performance(symbols, weights, period='1y'):
    df_portfolio = get_portfolio_history(symbols, period=period)
    expected_returns = df_portfolio.mean() * 252
    covariance_matrix = df_portfolio.cov() * 252
    weights = np.array(weights)
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    return portfolio_return, portfolio_volatility

df_portfolio = get_portfolio_history(symbols, period='1y')
def portfolio_performance_with_data(portfolio_df, weights, period='1y'):
    expected_returns = portfolio_df.mean() * 252
    covariance_matrix = portfolio_df.cov() * 252
    weights = np.array(weights)
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    return portfolio_return, portfolio_volatility

portfolio_return, portfolio_volatility = portfolio_performance_with_data(df_portfolio, weights, period='1y')
print(f"\nPortfolio({symbols}) Expected Annual Return: {portfolio_return:.2%}")
print(f"Portfolio({symbols}) Volatility (Risk): {portfolio_volatility:.2%}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Portfolio(['AAPL', 'NVDA', 'INTC']) Expected Annual Return: 28.12%
Portfolio(['AAPL', 'NVDA', 'INTC']) Volatility (Risk): 34.79%


## Getting best returns from yfinance

In [10]:

# 1. BIST Hisseleri Listesi (Sonuna .IS eklenmiş hali)
# Not: Buraya BIST100'ün tamamını da kopyalayabilirsin.
xu_tickers = [
    "THYAO.IS", "AKBNK.IS", "GARAN.IS", "ASELS.IS", "SISE.IS", 
    "KCHOL.IS", "SAHOL.IS", "TUPRS.IS", "BIMAS.IS", "EREGL.IS", 
    "FROTO.IS", "ISCTR.IS", "YKBNK.IS", "VAKBN.IS", "HALKB.IS",
    "PETKM.IS", "TCELL.IS", "ARCLK.IS", "ENKAI.IS", "TOASO.IS",
    "PGSUS.IS", "KOZAL.IS", "KRDMD.IS", "SASA.IS", "HEKTS.IS",
    "EKGYO.IS", "VESTL.IS", "ASTOR.IS", "REEDR.IS", "MGROS.IS"
]

def get_top_performers(tickers, period="1y"):
    print(f"{len(tickers)} hisse için veri indiriliyor...")
    
    # 2. Toplu Veri İndirme (Sadece 'Close' fiyatları yeterli)
    # threads=True işlemi hızlandırır
    data = yf.download(tickers, period=period, progress=False, auto_adjust=True)['Close']
    
    # 3. Getiri Hesaplama (Total Return)
    # Formül: (Son Fiyat / İlk Fiyat) - 1
    # iloc[0]: Dönem başı fiyatı, iloc[-1]: Dönem sonu (güncel) fiyat
    returns = (data.iloc[-1] / data.iloc[0]) - 1
    
    # 4. Sıralama ve İlk 10'u Seçme
    top_10 = returns.sort_values(ascending=False).head(10)
    
    return top_10, data

# --- UYGULAMA ---
top_gainers, all_data = get_top_performers(xu_tickers, period="1y")

print("-" * 40)
print("🏆 BIST - SON 1 YILIN EN ÇOK KAZANDIRANLARI")
print("-" * 40)

for symbol, ret in top_gainers.items():
    print(f"{symbol.replace('.IS', ''):<10} : %{ret*100:.2f}")



print("-" * 40)

30 hisse için veri indiriliyor...
----------------------------------------
🏆 BIST - SON 1 YILIN EN ÇOK KAZANDIRANLARI
----------------------------------------
ASELS      : %195.17
HALKB      : %119.18
KOZAL      : %65.26
EKGYO      : %59.22
ENKAI      : %58.94
TUPRS      : %44.06
VAKBN      : %28.41
TOASO      : %27.29
YKBNK      : %19.42
GARAN      : %15.73
----------------------------------------


In [11]:
# getting top 10 symbols
top_10_symbols = top_gainers.index.tolist()

print("Top 10 Symbols for Further Analysis:", top_10_symbols)

Top 10 Symbols for Further Analysis: ['ASELS.IS', 'HALKB.IS', 'KOZAL.IS', 'EKGYO.IS', 'ENKAI.IS', 'TUPRS.IS', 'VAKBN.IS', 'TOASO.IS', 'YKBNK.IS', 'GARAN.IS']


## Best Portfolio with Monte Carlo Simulation

In [12]:
import plotly.graph_objects as go

symbols = top_10_symbols
period = '1y'

df_portfolio = get_portfolio_history(symbols, period=period)

# ---  Simülasyon Ayarları ---
num_portfolios = 10000 # 10000 senaryo deneyelim
all_weights = np.zeros((num_portfolios, len(symbols)))
ret_arr = np.zeros(num_portfolios)
vol_arr = np.zeros(num_portfolios)
sharpe_arr = np.zeros(num_portfolios)

print(f"{num_portfolios} senaryo hesaplanıyor... for {period}")

# --- 3. Hızlı Döngü ---
for i in range(num_portfolios):
    # Rastgele ağırlıklar
    weights = np.random.random(len(symbols))
    weights = weights / np.sum(weights)
    all_weights[i, :] = weights
    
    # Performans Hesapla
    ret, vol = portfolio_performance_with_data(df_portfolio, list(weights), period='1y')
    
    ret_arr[i] = ret
    vol_arr[i] = vol
    sharpe_arr[i] = ret / vol

# --- 4. En İyisini Bul ---
max_sharpe_idx = sharpe_arr.argmax()
best_ret = ret_arr[max_sharpe_idx]
best_vol = vol_arr[max_sharpe_idx]
best_w = all_weights[max_sharpe_idx, :]

print("-" * 30)
print(f"EN İYİ PORTFÖY (Max Sharpe: {sharpe_arr[max_sharpe_idx]:.2f})")
print(f"Getiri: %{best_ret:.2%}")
print(f"Risk:   %{best_vol:.2%}")
print("DAĞILIM:")
for i, s in enumerate(symbols):
    print(f"{s}: %{best_w[i]:.2%}")
print("-" * 30)


# --- 5. Görselleştirme ---
fig = go.Figure()

# Tüm Senaryolar
fig.add_trace(go.Scatter(
    x=vol_arr, y=ret_arr,
    mode='markers',
    marker=dict(
        size=5,
        color=sharpe_arr,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Sharpe Ratio")
    ),
    text=[f"Sharpe: {s:.2f}" for s in sharpe_arr],
    name='Olası Portföyler'
))

# En İyi Nokta
fig.add_trace(go.Scatter(
    x=[best_vol], y=[best_ret],
    mode='markers',
    marker=dict(color='red', size=15, symbol='star'),
    name='OPTIMAL PORTFÖY'
))

fig.update_layout(
    title=f'{symbols} Efficient Frontier Analizi',
    xaxis_title='Risk (Volatilite)',
    yaxis_title='Getiri (Return)',
    height=600
)

fig.show()

print("Bitti.")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


10000 senaryo hesaplanıyor... for 1y
------------------------------
EN İYİ PORTFÖY (Max Sharpe: 2.37)
Getiri: %62.54%
Risk:   %26.40%
DAĞILIM:
ASELS.IS: %24.86%
HALKB.IS: %14.03%
KOZAL.IS: %8.83%
EKGYO.IS: %12.29%
ENKAI.IS: %14.49%
TUPRS.IS: %14.37%
VAKBN.IS: %3.60%
TOASO.IS: %5.24%
YKBNK.IS: %0.55%
GARAN.IS: %1.74%
------------------------------


Bitti.


## Risk Analyis